In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from category_encoders import CatBoostEncoder

In [ ]:
def rellenar_con_X_por_grupo(df, grupo, rellenar, func):
    valoresAgrupados = df.groupby(grupo)[rellenar].transform(func)
    df[rellenar].fillna(valoresAgrupados, inplace=True)

In [ ]:
df = pd.read_csv("/kaggle/input/tp2datostrain/train.csv");

In [ ]:
df.columns

In [ ]:
df.dtypes

In [ ]:
print("-Nulos garages: ", df.garages.isna().value_counts())
print("-Nulos gim: ", df.gimnasio.isna().value_counts())
print("-Nulos SUM: ", df.usosmultiples.isna().value_counts())
print("-Nulos piscina: ", df.piscina.isna().value_counts())
print("-Nulos escuelas: ", df.escuelascercanas.isna().value_counts())
print("-Nulos Centro comerc: ", df.centroscomercialescercanos.isna().value_counts())

In [ ]:
#RELLENO LOS IDS CON LO QUE MAS APARECE EN LA CIUDAD
def obtenerMasFrecuente(x):
    x = pd.Series.mode(x)
    s = x.loc[x>0]
    if len(s.index)<=0:
        return np.nan
    return s.value_counts().index[0]
print("Nulos idzona: ", df["idzona"].isna().value_counts())
print("Nulos coidad: ", df["ciudad"].isna().value_counts())

aux = df[["ciudad", "idzona"]].copy()
aux["ciudad"].fillna("dummy", inplace=True)
#aux["idzona"].fillna(-1, inplace=True)
idzonaMasFrecuentePorCiudad = aux.groupby('ciudad')["idzona"].transform(lambda x: obtenerMasFrecuente(x))
idzonaMasFrecuentePorCiudad

In [ ]:
df.idzona.fillna(idzonaMasFrecuentePorCiudad, inplace = True)
print("Nulos idzona: ", df["idzona"].isna().value_counts())
#DROPEO LAS COLUMNAS QUE QUEDARON SIN NADA
df.dropna(subset=['idzona'], inplace = True)
print("Nulos idzona: ", df["idzona"].isnull().value_counts())
df.shape

In [ ]:
#RELLENO CON NULOS LOS GARAGES
df.garages.fillna(0, inplace = True)
#df.gimnasio.fillna(0, inplace = True)
#df.usosmultiples.fillna(0, inplace = True)
#df.piscina.fillna(0, inplace = True)
#df.escuelascercanas.fillna(0, inplace = True)
#df.centroscomercialescercanos.fillna(0, inplace = True)

In [ ]:
#Reemplazo las habitaciones por la mediana de la zona
print("Nulos habitaciones: ", df["habitaciones"].isnull().value_counts())
rellenar_con_X_por_grupo(df, "idzona", "habitaciones", lambda x: x.median())
print("Nulos habitaciones: ", df["habitaciones"].isnull().value_counts())
#Relleno el resto con las mas frecuentes
df['habitaciones'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['habitaciones']])

In [ ]:
df.drop(columns = 'id', inplace = True)
#Saco el id porque es unico para cada propriedad

In [ ]:
#Para la antiguedad relleno con el promedio de la zona
print("Nulos antiguedad: ", df["antiguedad"].isna().value_counts())
medianaAntiguedadXZona = df.groupby("idzona")["antiguedad"].transform(lambda x: x.median())
df["antiguedad"].fillna(medianaAntiguedadXZona, inplace = True)
print("Nulos antiguedad: ", df["antiguedad"].isna().value_counts())

Quedaron algunos nulos en la antiguedad

In [ ]:
#Reemplazo los banos por 1 que es lo minimo posible
df.banos.fillna(0, inplace=True)

In [ ]:
#hago el swap PARA LOS METROS
df["swapear"] = df["metrostotales"] < df["metroscubiertos"]
df_metros_arreglados = df.copy()
aux = df_metros_arreglados.loc[df_metros_arreglados["swapear"], "metrostotales"]
if (not aux.empty):
    df_metros_arreglados.loc[df_metros_arreglados["swapear"],"metrostotales"] = \
    df_metros_arreglados.loc[df_metros_arreglados["swapear"], "metroscubiertos"]
    df_metros_arreglados.loc[df_metros_arreglados["swapear"], "metroscubiertos"] = aux
df_metros_arreglados.drop(columns="swapear", inplace=True)
(df_metros_arreglados["metrostotales"] < df_metros_arreglados["metroscubiertos"]).value_counts()
df = df_metros_arreglados
print("Nulos metrostotales: ", df["metrostotales"].isna().value_counts())
print("Nulos metroscubiertos: ", df["metroscubiertos"].isna().value_counts())
#COMO QUEDAN NULOS LOS RELLENO CON la mediana (porque la mediana es un valor, y es menos sensible a valores extremos) por ZONA / CIUDAD
promedioMetrosCubiertosXZona = df.groupby("idzona")["metroscubiertos"].transform(lambda x: x.median())
df["metroscubiertos"].fillna(promedioMetrosCubiertosXZona, inplace = True)
print("Nulos metroscubiertos: ", df["metroscubiertos"].isna().value_counts())
promedioMetrosTotalesXZona = df.groupby("idzona")["metrostotales"].transform(lambda x: x.median())
df["metrostotales"].fillna(promedioMetrosCubiertosXZona, inplace = True)
print("Nulos metrostotales: ", df["metrostotales"].isna().value_counts())


Quedaron algunos metros cubiertos nulos! Podría ser porque hay algunas propiedades que son al aire libre. vemos depues si:
- las rellenamos con un promedio de otra cosa o 
- con 0, o 
- si las tiramos

In [ ]:
df.drop(['lat', 'lng', 'titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad', 'provincia', 'fecha'], axis=1, inplace=True)
#Temporalmente dropeo todo lo que quedo con nulos
df.dropna(inplace = True)
df.columns

In [ ]:
features = df.drop('precio', axis=1).columns
X_train,X_test,y_train,y_test = train_test_split(df[features], df.precio,test_size=0.3, random_state = 0)

In [ ]:
clf = RandomForestClassifier(n_estimators = 40, max_depth = 8)
clf.fit(X_train, y_train)

In [ ]:
import matplotlib.pyplot as plt
impor = np.vstack((features, clf.feature_importances_)).T
imporDf = pd.DataFrame(impor)
imporDf.columns = ['feature', 'importancia']
imporDf = imporDf.set_index('feature').sort_values(by=['importancia'], ascending=False).iloc[::-1]
plt.figure(figsize=(20,20))
ax = imporDf.plot(kind='barh')
ax.set_title("Importancia de cada feature segun Random Forests")
ax.set(xlabel="Importancia", ylabel="Feature")



In [ ]:
preds = clf.predict(X_test)

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, preds)

In [ ]:
x = df.drop(columns = ['id', 'titulo', 'descripcion', 'direccion', 'ciudad', 'lat', 'lng']).copy()

In [ ]:
x.metrostotales.fillna(value='0', inplace = True)
x.metroscubiertos.fillna(value='0', inplace = True)
x.metrostotales = x.metrostotales.astype(int)
x.metroscubiertos = x.metroscubiertos.astype(int)
x['metrostotales'] = x[['metrostotales', 'metroscubiertos']].max(axis=1)
x['metroscubiertos'] = x[['metrostotales', 'metroscubiertos']].min(axis=1)
x['patio'] = x['metrostotales'] - x['metroscubiertos']

In [ ]:
x['fecha'] = pd.to_datetime(x['fecha'])
x['anio'] = x.fecha.dt.year
x['mes'] = x.fecha.dt.month
x['dia'] = x.fecha.dt.day
x = x.drop(columns = 'fecha')

In [ ]:
transformador_numeros = SimpleImputer(fill_value = 0)
#transformador_catboost = Pipeline(steps=[('imputer1', SimpleImputer(strategy='most_frequent')),
#                                           ('catboost', ce.CatBoostEncoder(handle_unknown='ignore'))])
transformador_onehot = Pipeline(steps=[('imputer2', SimpleImputer(strategy='most_frequent')),
                                        ('onehot', OneHotEncoder(handle_unknown='ignore'))])
cols_numericas = ['antiguedad', 'habitaciones','garages', 'banos', 'metroscubiertos',
                  'metrostotales', 'idzona', 'gimnasio', 'usosmultiples',
                  'piscina', 'escuelascercanas','centroscomercialescercanos', 'patio', 'anio', 'mes', 'dia']
#cols_catboost = ['ciudad']
cols_onehot = ['tipodepropiedad', 'provincia']

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', transformador_numeros, cols_numericas),
#        ('cat', transformador_catboost, cols_catboost),
        ('one', transformador_onehot, cols_onehot)
    ])

In [ ]:
features = x.drop('precio', axis=1).columns
X_train,X_test,y_train,y_test = train_test_split(x[features], x.precio,test_size=0.3, random_state = 0)
X_train.columns

In [ ]:
clf = RandomForestClassifier(n_estimators=10000, random_state=0)
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', clf)
                             ])

In [ ]:
my_pipeline.fit(X_train,y_train)
#X_train.columns

In [ ]:
a = SimpleImputer()
x[cols_numericas] = a.fit_transform(x[cols_numericas])

In [ ]:
x.dtypes